# Error Correcting Codes Encoding Study

The goal of this study is to understand options to the popular one-hot encoding. There are many sides to each story (no, not only two), on those sides are: 

- I never liked one-hot encoding (and is been more than a decade since I first used it, so the disgust might never go out);
- I don't like how neural networks are treated and should always be end to end learning (no they should not, they should be more complex architectures, many already in research literature)
- There are priors 
- Each type of input should  (and HAS in nature) its own priors, which are adapted to *facilitate* the learning, no we should not do everyhting inside a NN, we should give as input something that has priors that facilitate learning (and might or might not later save processing power during operations)


On the priors, many have already shown good results, the most remarcable prior are: Convolutional Neural Networks, MAC Networks, LSTMs, others are more subtle, like (remember citation here ...) adding a coordinate system to the input image as an (or many) extra channel(s). There are many more that I think are worth exploring and adding to the literature, even if they don't give good results. 
On those priors there are many that we not only know, but also we have specialized hardware that is perfectly adapted
* time and space -> this we can encode and add it as extra channels
* Different transforms (Fourier, Laplace, Wavelets, ...)
* spikes (borders in images)
* ....


The idea  behind this is that I don't agree with one-hot encoding, not because it does not work, but because it imposes a few limits that I don't want to deal with at first

* We know the actual number of values to encode (with words this is not necessary true)
* We have a sample data to train the encoding

This limits us in several ways; for example, for training on a domain, the encoder will depend on that domain only. If there are under-represented values (such as words that don't appear or are new later, or changing domain) this limits the encoding possibliities. A better idea will be to be able to encode everything even if the internal representations have not yet learned to use those simbols.

I want to be able to do a separation ebtween the *possibility*  of representing a value, and the learning of that concept.

The first and biggest limitation of one-hot encoding is that does not allow to represent values that are not accepted.

As some other parts of this study have already focused on integer value representations, arbitrary function representation (although with limitted success on the fourier inspired encodings) this study is more focused on being able to represent correctly all the values of utf-8, basically doing a first binary representation that will be given as input to an OVERFITTED encoder. 

The reasoning behind this is:


* The origin domain is all text
* UTF-8 can represent all text in all languages including some extra elements
* let's use UTF-8 as the origin domain
* Create an encoder that can deal with ANY and ALL input in the origin domain
* the encoded values can later be used

As text should be always correctly reconstructed in the presence of noise, I want to imagine now a Neural Network like a REALLY NOISY channel. For this using (Forward) ECCs is one way of thinking in this medium
Then the tests that I intend to do is:

* Create an autoencoder that OVERFITS to all the data


One idea that I have been dealing with my head for the past 3-4 years is that we are thinking overfitting the wrong way, and we can actually use it well, but we have to learn how.

I think that here is the first time I actually find a way of doing it in a useful way

The idea is to overfit so it generates an smaller encoding vector than the one in the input. Autoencoders are good to do this test.

The other idea is that if the autoencoder can NOT do this, then the encoding ideas that I will try are BAAAD and I should feel BAAAD. In this case ... just go to the drrawing table and think of other things.

On the other side, if this works, this means that FINALLY I can go on the next stage, that is building the predictors first basic ones (LSTMs, HMMs, Time Convolutions), then with meta-learning and later with my still too fresh idea on neural databases. 

One interesting thing I want to find out about Error Correcting Codes (ECCs) is if they are actually useful in the output decoding, as they should be adding *explicit* redundancy to the input and also to the output.

The other thing about ECCs is that we can pile them up, for example, one (or many codes) to representa a symbol (for example the value *'€'* ) and then use convolutional or turbo codes for the *temporal* encoding/decoding part, this means that we not only add priors to the intantaneous input, but also to the temporal dimension, which is something I really want to explore (and this should facilitate fixing and correcting "channel errors")

I don't deal here with *erasure* error types, but that is a possibility later.


In [ ]:
import numpy as np